Predicict how long Ukhamba - Inkabi Zezwe  would take to reach gold status based on RISA certification standards as listed below. I will train a Machine Learning algorithm using histrical data of 
the artists

In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error

: 

In [ ]:

# Read the data into a Pandas DataFrame
data = pd.read_csv('streaming_data.csv', )